# Exploring Dandiset 000690: Allen Institute Openscope - Vision2Hippocampus project

> **DISCLAIMER**: This notebook was AI-generated and has not been fully verified. Please exercise caution when interpreting the code or results, and verify important findings independently.

## Overview

The [Allen Institute Openscope - Vision2Hippocampus project](https://dandiarchive.org/dandiset/000690) investigates how visual information is processed and transformed as it travels from the thalamus through visual cortical areas to the hippocampus in mice. This project aims to understand how neural representations of simple and natural visual stimuli evolve across this processing pathway.

In this notebook, we will:

1. Explore the structure of the Dandiset
2. Examine electrophysiological data from a Neuropixels probe recording
3. Analyze neural spiking activity across brain regions
4. Investigate neural responses to various visual stimuli
5. Visualize the relationship between visual stimuli and neural activity

## Required Packages

This notebook requires the following Python packages:

- pynwb
- h5py
- remfile
- numpy
- matplotlib
- pandas
- seaborn

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import h5py
import remfile
import pynwb
import pandas as pd
import seaborn as sns

# Set seaborn style for most plots
sns.set_theme(style="whitegrid")

## Accessing the Dandiset

We'll use the DANDI API to access the Dandiset and explore its contents.

In [ ]:
from dandi.dandiapi import DandiAPIClient

# Connect to DANDI archive
client = DandiAPIClient()
dandiset = client.get_dandiset("000690")

# Print basic information about the Dandiset
metadata = dandiset.get_raw_metadata()
print(f"Dandiset name: {metadata['name']}")
print(f"Dandiset URL: {metadata['url']}")

# List the assets in the Dandiset
assets = list(dandiset.get_assets())
print(f"\nFound {len(assets)} assets in the dataset")
print("\nFirst 5 assets:")
for asset in assets[:5]:
    print(f"- {asset.path}")

## Dataset Structure

This Dandiset contains extracellular electrophysiology recordings using Neuropixels probes in mice. The recordings were performed while presenting different visual stimuli to the mice.

The main file types include:

1. **Main session files** (e.g., `sub-692072_ses-1298465622.nwb`) - Contains metadata, units (spikes), and other session information
2. **Image files** (e.g., `sub-692072_ses-1298465622_image.nwb`) - Contains visual stimulus templates
3. **Probe-specific ecephys files** (e.g., `sub-692072_ses-1298465622_probe-0_ecephys.nwb`) - Contains LFP data for specific probes

Let's first examine a probe's LFP data to understand brain activity during the experiment.

## Exploring LFP Data from a Neuropixels Probe

Local Field Potentials (LFPs) represent the summed synaptic activity of neurons near the recording electrode. Let's load LFP data from one of the probes and examine it.

In [ ]:
# URL for probe 0 data from subject 692072
probe_url = "https://api.dandiarchive.org/api/assets/ba8760f9-91fe-4c1c-97e6-590bed6a783b/download/"
print(f"Neurosift link: https://neurosift.app/nwb?url={probe_url}&dandisetId=000690&dandisetVersion=draft")

# Load the data
print("Loading data from remote NWB file...")
remote_file = remfile.File(probe_url)
h5_file = h5py.File(remote_file)
io = pynwb.NWBHDF5IO(file=h5_file)
nwb = io.read()

print(f"Session ID: {nwb.session_id}")
print(f"Session description: {nwb.session_description}")

### Electrode Information

Let's examine the electrodes used in this recording, including their positions and associated brain regions.

In [ ]:
# Get electrode information
electrodes_df = nwb.electrodes.to_dataframe()
print(f"Number of electrodes: {len(electrodes_df)}")
print("\nSample of electrode data:")
print(electrodes_df.head())

# Get information about brain regions represented
brain_regions = electrodes_df['location'].unique()
print(f"\nBrain regions in recording: {brain_regions}")

# Plot brain region counts
plt.figure(figsize=(12, 6))
electrodes_df['location'].value_counts().plot(kind='bar')
plt.title('Number of Electrodes per Brain Region')
plt.xlabel('Brain Region')
plt.ylabel('Count')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()

# Plot electrode positions
plt.figure(figsize=(10, 8))
plt.scatter(
    electrodes_df['probe_horizontal_position'], 
    electrodes_df['probe_vertical_position'],
    c=electrodes_df.index, 
    cmap='viridis', 
    alpha=0.8,
    s=50
)
plt.colorbar(label='Electrode index')
plt.xlabel('Horizontal position (µm)')
plt.ylabel('Vertical position (µm)')
plt.title('Probe electrode positions')
plt.grid(True)
plt.show()

### LFP Data

Now let's look at the structure of the LFP data and visualize some example traces.

In [ ]:
# Get access to LFP data
probe_0_lfp = nwb.acquisition['probe_0_lfp']
probe_0_lfp_data = probe_0_lfp.electrical_series['probe_0_lfp_data']

print(f"LFP data shape: {probe_0_lfp_data.data.shape}")
print(f"LFP time points: {len(probe_0_lfp_data.timestamps)}")
print(f"LFP sampling rate: {1/(probe_0_lfp_data.timestamps[1] - probe_0_lfp_data.timestamps[0])} Hz")
print(f"LFP duration: {probe_0_lfp_data.timestamps[-1] - probe_0_lfp_data.timestamps[0]} seconds")

Let's visualize some LFP traces from different brain regions to see regional differences in activity.

In [ ]:
# Plot a sample of LFP data from different channels
# Sample 1 second of data starting at an interesting point
start_time = 10000  # Starting sample
sample_length = 1250  # 1 second at 1250 Hz

# Sample every Nth channel to get a representative set
N = 20  # Take every 20th channel
sampled_channels = list(range(0, probe_0_lfp_data.data.shape[1], N))[:5]  # Take up to 5 channels

# Get sample data for selected channels
sample_data = probe_0_lfp_data.data[start_time:start_time+sample_length, sampled_channels]
sample_time = probe_0_lfp_data.timestamps[start_time:start_time+sample_length]

# Plot sample LFP traces
plt.figure(figsize=(15, 10))
for i, channel_idx in enumerate(sampled_channels):
    # Offset each trace for visibility
    offset = i * 0.0005  # Adjust based on data amplitude
    plt.plot(
        sample_time, 
        sample_data[:, i] + offset, 
        label=f'Channel {channel_idx}'
    )

plt.xlabel('Time (s)')
plt.ylabel('LFP (V)')
plt.title('Sample LFP traces from different channels')
plt.legend()
plt.grid(True)
plt.show()

Let's also visualize the LFP activity across all channels as a heatmap to get a better sense of the spatial patterns.

In [ ]:
# Create a heatmap of LFP activity across channels
plt.figure(figsize=(15, 8))
plt.imshow(
    sample_data.T, 
    aspect='auto',
    extent=[sample_time[0], sample_time[-1], 0, len(sampled_channels)-1],
    origin='lower', 
    cmap='viridis'
)
plt.colorbar(label='LFP (V)')
plt.xlabel('Time (s)')
plt.ylabel('Channel index')
plt.title('LFP activity across selected channels')
plt.yticks(range(len(sampled_channels)), [f'Ch {ch}' for ch in sampled_channels])
plt.show()

## Exploring Spiking Activity

Now let's examine the spiking activity (units) from the main session file.

In [ ]:
# URL for the main session file
session_url = "https://api.dandiarchive.org/api/assets/fbcd4fe5-7107-41b2-b154-b67f783f23dc/download/"
print(f"Neurosift link: https://neurosift.app/nwb?url={session_url}&dandisetId=000690&dandisetVersion=draft")

# Load the data
print("Loading session data from remote NWB file...")
remote_file = remfile.File(session_url)
h5_file = h5py.File(remote_file)
io = pynwb.NWBHDF5IO(file=h5_file)
session_nwb = io.read()

print(f"Session ID: {session_nwb.session_id}")
print(f"Session description: {session_nwb.session_description}")

### Unit Information

Let's explore the properties of the recorded units (neurons).

In [ ]:
# Get units data
units_df = session_nwb.units.to_dataframe()
print(f"Number of units: {len(units_df)}")
print("\nSample of units data:")
print(units_df[['firing_rate', 'quality', 'isi_violations']].head())

# Plot firing rate distribution
plt.figure(figsize=(10, 6))
sns.histplot(units_df['firing_rate'].clip(upper=50), bins=50, kde=True)
plt.xlabel('Firing Rate (Hz, clipped at 50 Hz)')
plt.ylabel('Count')
plt.title('Distribution of Neuron Firing Rates')
plt.show()

# Plot quality metrics distribution (if available)
if 'quality' in units_df.columns and units_df['quality'].dtype != object:
    plt.figure(figsize=(10, 6))
    sns.countplot(x='quality', data=units_df)
    plt.title('Distribution of Unit Quality')
    plt.xlabel('Quality')
    plt.ylabel('Count')
    plt.show()

### Spike Raster Plot

Let's create a raster plot to visualize spike times for a few example units.

In [ ]:
# Select a few units with different firing rates
units_by_firing_rate = units_df.sort_values('firing_rate', ascending=False)
sample_units = units_by_firing_rate.iloc[[0, 10, 50, 100, 250]].index

# Get spike times for sample units
sample_spike_times = {}
for unit_id in sample_units:
    spike_times = units_df.loc[unit_id, 'spike_times']
    # Only keep spike times in a reasonable range to visualize
    # For example, 10 seconds of data starting at time 100
    sample_spike_times[unit_id] = spike_times[(spike_times >= 100) & (spike_times < 110)]

# Plot the raster
plt.figure(figsize=(15, 8))
for i, (unit_id, spike_times) in enumerate(sample_spike_times.items()):
    plt.scatter(
        spike_times, 
        np.ones_like(spike_times) * i, 
        s=10, 
        label=f"Unit {unit_id} (FR: {units_df.loc[unit_id, 'firing_rate']:.2f} Hz)"
    )

plt.xlabel('Time (s)')
plt.yticks(range(len(sample_units)), [f"Unit {id}" for id in sample_units])
plt.ylabel('Unit')
plt.title('Spike Raster Plot for Sample Units (10 sec window)')
plt.grid(True, axis='x')
plt.show()

## Visual Stimuli

Now let's examine the visual stimuli used in the experiment. The stimuli are stored in the image NWB file.

In [ ]:
# URL for the image file
image_url = "https://api.dandiarchive.org/api/assets/cbc64387-19b9-494a-a8fa-04d3207f7ffb/download/"
print(f"Neurosift link: https://neurosift.app/nwb?url={image_url}&dandisetId=000690&dandisetVersion=draft")

# Load the data
print("Loading image data from remote NWB file...")
remote_file = remfile.File(image_url)
h5_file = h5py.File(remote_file)
io = pynwb.NWBHDF5IO(file=h5_file)
image_nwb = io.read()

### Stimulus Templates

Let's look at the stimulus templates available in the dataset.

In [ ]:
# Look at the stimulus template section
stimulus_templates = image_nwb.stimulus_template
print(f"Available stimulus templates: {list(stimulus_templates.keys())}")

The stimuli in this dataset fall into several categories:

1. **SAC (and variants)**: Simple moving bars with different parameters
2. **Disco2SAC**: Colorful disco-like moving bars
3. **Ring**: Ring-shaped stimuli
4. **Disk**: Disk-shaped stimuli
5. **natmovie**: Natural movies of various scenes

The stimuli are parameterized with properties like:
- **Wd**: Width (e.g., 15 or 45 degrees)
- **Vel**: Velocity (e.g., 2 or 8 degrees/second)
- **Bndry**: Boundary conditions for the stimulus
- **Cntst**: Contrast level

Let's visualize examples from different stimulus categories.

In [ ]:
# Function to display frames from a stimulus
def display_stimulus_frames(stim_name, num_frames=4):
    if stim_name in stimulus_templates:
        stim_template = stimulus_templates[stim_name]
        print(f"\nStimulus: {stim_name}")
        print(f"Shape: {stim_template.data.shape}")
        print(f"Rate: {stim_template.rate} Hz")
        
        # Determine if it's a RGB stimulus or grayscale
        is_rgb = len(stim_template.data.shape) == 4

        # Get frames at regular intervals
        if is_rgb:
            n_frames = stim_template.data.shape[2]
            frame_indices = np.linspace(0, n_frames-1, num_frames, dtype=int)
            
            fig, axes = plt.subplots(1, num_frames, figsize=(16, 4))
            for i, frame_idx in enumerate(frame_indices):
                frame = stim_template.data[:, :, frame_idx, :]
                axes[i].imshow(frame)
                axes[i].set_title(f"Frame {frame_idx}")
                axes[i].axis('off')
        else:
            n_frames = stim_template.data.shape[2]  
            frame_indices = np.linspace(0, n_frames-1, num_frames, dtype=int)
            
            fig, axes = plt.subplots(1, num_frames, figsize=(16, 4))
            for i, frame_idx in enumerate(frame_indices):
                frame = stim_template.data[:, :, frame_idx]
                axes[i].imshow(frame, cmap='gray')
                axes[i].set_title(f"Frame {frame_idx}")
                axes[i].axis('off')
                
        plt.suptitle(f"{stim_name.replace('_presentations', '')}")
        plt.tight_layout()
        plt.show()
    else:
        print(f"Stimulus template not found for {stim_name}")

# Display example stimuli
example_stimuli = [
    "SAC_Wd15_Vel2_Bndry1_Cntst0_loop_presentations",  # Standard bar stimulus
    "Disco2SAC_Wd15_Vel2_Bndry1_Cntst0_loop_presentations",  # Disco bar stimulus
    "SAC_Wd45_Vel2_Bndry1_Cntst0_loop_presentations",  # Wide bar stimulus
    "natmovie_EagleSwooping1_540x960Full_584x460Active_presentations"  # Natural movie
]

for stim_name in example_stimuli:
    display_stimulus_frames(stim_name)

### Stimulus Presentations

Now let's examine when these stimuli were presented during the experiment.

In [ ]:
# Look at presentation intervals
interval_names = list(session_nwb.intervals.keys())
print(f"Number of stimulus presentation intervals: {len(interval_names)}")
print(f"Sample of interval names: {interval_names[:5]}")

# Group stimulus names by pattern
stimulus_types = {}
for name in interval_names:
    if "_presentations" not in name:
        continue
    
    base_name = name.replace("_presentations", "")
    parts = base_name.split("_")
    
    # Group by first part of name (stimulus type)
    stim_type = parts[0]
    if stim_type not in stimulus_types:
        stimulus_types[stim_type] = []
    
    stimulus_types[stim_type].append(name)

# Print stimulus categories
print("\nStimulus categories:")
for stim_type, intervals in stimulus_types.items():
    print(f"- {stim_type}: {len(intervals)} variants")

# Plot number of variants per stimulus type
plt.figure(figsize=(12, 6))
counts = [len(intervals) for stim_type, intervals in stimulus_types.items()]
plt.bar(stimulus_types.keys(), counts)
plt.xlabel('Stimulus Type')
plt.ylabel('Number of Variants')
plt.title('Number of Stimulus Variants per Category')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()

Let's examine the timing of stimulus presentations for one example stimulus type.

In [ ]:
# Get presentation intervals for a sample stimulus
sample_stim_name = next(iter(stimulus_types['SAC']))
presentation_df = session_nwb.intervals[sample_stim_name].to_dataframe()

print(f"\nSample of stimulus presentation intervals for {sample_stim_name}:")
print(presentation_df.head())

# Plot the presentation times
plt.figure(figsize=(12, 6))
plt.scatter(presentation_df['start_time'], np.zeros_like(presentation_df['start_time']), 
            alpha=0.5, marker='|', s=80)
plt.xlabel('Time (s)')
plt.yticks([])
plt.title(f'Presentation Times for {sample_stim_name.replace("_presentations", "")}')
plt.xlim(presentation_df['start_time'].min(), 
         min(presentation_df['start_time'].min() + 300, presentation_df['start_time'].max()))
plt.grid(True, axis='x')
plt.show()

# Plot distribution of stimulus durations
plt.figure(figsize=(10, 6))
durations = presentation_df['stop_time'] - presentation_df['start_time']
sns.histplot(durations, bins=50, kde=True)
plt.xlabel('Duration (s)')
plt.ylabel('Count')
plt.title(f'Distribution of Stimulus Durations for {sample_stim_name.replace("_presentations", "")}')
plt.show()

## Analyzing Neural Responses to Visual Stimuli

Now let's analyze how neurons respond to the visual stimuli. We'll look at a simple example of spike-triggered averaging.

In [ ]:
# Function to compute peri-stimulus time histogram (PSTH)
def compute_psth(spike_times, event_times, window=(-0.5, 1.5), bin_size=0.01):
    """
    Compute peri-stimulus time histogram for a given unit around stimulus events.
    
    Args:
        spike_times: spike times for a single unit
        event_times: stimulus presentation times
        window: time window around each event (in seconds)
        bin_size: bin size for histogram (in seconds)
    
    Returns:
        bins: bin centers for the histogram
        psth: peri-stimulus time histogram
    """
    # Create bins
    bins = np.arange(window[0], window[1] + bin_size, bin_size)
    bin_centers = bins[:-1] + bin_size/2
    
    # Initialize PSTH
    psth = np.zeros(len(bin_centers))
    
    # For each event, find spikes within the window
    for event_time in event_times:
        # Find spikes relative to this event
        relative_spikes = spike_times - event_time
        # Filter to spikes within the window
        mask = (relative_spikes >= window[0]) & (relative_spikes < window[1])
        filtered_spikes = relative_spikes[mask]
        
        # Bin the spikes
        hist, _ = np.histogram(filtered_spikes, bins=bins)
        
        # Add to PSTH
        psth += hist
    
    # Normalize by number of events and bin size to get firing rate
    psth = psth / (len(event_times) * bin_size)
    
    return bin_centers, psth

Let's select a stimulus and compute PSTHs for a few units to see how they respond to the stimulus.

In [ ]:
# Choose a specific stimulus type
stim_name = sample_stim_name  # Use the same stimulus from above

# Get stimulus presentation times
stim_times = session_nwb.intervals[stim_name].to_dataframe()['start_time'].values

# Select a few units to analyze
high_fr_units = units_by_firing_rate.head(5).index

# Compute and plot PSTHs
fig, axes = plt.subplots(len(high_fr_units), 1, figsize=(12, 10), sharex=True)

for i, unit_id in enumerate(high_fr_units):
    spike_times = units_df.loc[unit_id, 'spike_times']
    
    # Compute PSTH
    bin_centers, psth = compute_psth(spike_times, stim_times, window=(-0.2, 1.0), bin_size=0.01)
    
    # Plot PSTH
    axes[i].bar(bin_centers, psth, width=0.01, alpha=0.7)
    axes[i].axvline(x=0, color='r', linestyle='--', alpha=0.6)  # Mark stimulus onset
    axes[i].set_ylabel('Firing Rate (Hz)')
    axes[i].set_title(f'Unit {unit_id} (Firing Rate: {units_df.loc[unit_id, "firing_rate"]:.2f} Hz)')
    
    # Set ylim to better visualize responses
    if psth.max() > 0:
        axes[i].set_ylim(0, psth.max() * 1.2)

# Add labels
axes[-1].set_xlabel('Time from Stimulus Onset (s)')
plt.suptitle(f'Neural Responses to {stim_name.replace("_presentations", "")}', fontsize=16)
plt.tight_layout()
plt.show()

## Analyzing Responses to Different Stimulus Types

Let's compare how a single unit responds to different types of stimuli.

In [ ]:
# Choose a unit that had a strong response
unit_id = high_fr_units[0]  # Choose the unit with the highest firing rate

# Choose different stimulus types
stim_types = ['SAC', 'Disco2SAC', 'natmovie']
sample_stim_names = []

for stim_type in stim_types:
    if stim_type in stimulus_types and len(stimulus_types[stim_type]) > 0:
        sample_stim_names.append(stimulus_types[stim_type][0])

# Compute and plot PSTHs for each stimulus type
fig, axes = plt.subplots(len(sample_stim_names), 1, figsize=(12, 10), sharex=True)

for i, stim_name in enumerate(sample_stim_names):
    # Get stimulus presentation times
    stim_times = session_nwb.intervals[stim_name].to_dataframe()['start_time'].values
    
    # Get spike times for the selected unit
    spike_times = units_df.loc[unit_id, 'spike_times']
    
    # Compute PSTH
    bin_centers, psth = compute_psth(spike_times, stim_times, window=(-0.2, 1.0), bin_size=0.01)
    
    # Plot PSTH
    axes[i].bar(bin_centers, psth, width=0.01, alpha=0.7)
    axes[i].axvline(x=0, color='r', linestyle='--', alpha=0.6)  # Mark stimulus onset
    axes[i].set_ylabel('Firing Rate (Hz)')
    axes[i].set_title(f'Response to {stim_name.replace("_presentations", "")}')
    
    # Set ylim to better visualize responses
    if psth.max() > 0:
        axes[i].set_ylim(0, psth.max() * 1.2)

# Add labels
axes[-1].set_xlabel('Time from Stimulus Onset (s)')
plt.suptitle(f'Unit {unit_id} Responses to Different Stimuli', fontsize=16)
plt.tight_layout()
plt.show()

## Summary and Conclusions

In this notebook, we explored Dandiset 000690 from the Allen Institute Openscope - Vision2Hippocampus project. This dataset contains recordings from multiple Neuropixels probes in mice, capturing neural activity across various brain regions while presenting different visual stimuli.

Key findings:

1. The dataset includes recordings from multiple brain regions, including visual cortex, thalamus, and hippocampus.
2. The LFP signals show distinct patterns across different brain regions, reflecting regional differences in neural activity.
3. Units (neurons) exhibit a wide range of firing rates, with most neurons firing at low rates (< 5 Hz).
4. The dataset includes an extensive set of visual stimuli, including simple moving bars with various parameters, disco-colored bars, and natural movies.
5. Neural responses to visual stimuli show interesting temporal patterns, with some units exhibiting clear stimulus-locked responses.

### Future Directions

This notebook provides a starting point for more in-depth analyses of this rich dataset. Possible future directions include:

1. Comparing neural responses across different brain regions to understand how visual information is transformed along the processing pathway.
2. Analyzing how specific stimulus parameters (width, velocity, contrast) affect neural responses.
3. Comparing responses to artificial stimuli versus natural movies to understand how the brain processes natural scenes differently.
4. Examining synchronization between brain regions during stimulus presentation.
5. Applying advanced analyses like dimensionality reduction or encoding models to understand population-level representations of visual information.